# Sentiment Analysis with Deep Learning using BERT

### Prerequisites

- Intermediate-level knowledge of Python 3 (NumPy and Pandas preferably, but not required)
- Exposure to PyTorch usage
- Basic understanding of Deep Learning and Language Models (BERT specifically)

### Project Outline

**Task 1**: Introduction (this section)

**Task 2**: Exploratory Data Analysis and Preprocessing

**Task 3**: Training/Validation Split

**Task 4**: Loading Tokenizer and Encoding our Data

**Task 5**: Setting up BERT Pretrained Model

**Task 6**: Creating Data Loaders

**Task 7**: Setting Up Optimizer and Scheduler

**Task 8**: Defining our Performance Metrics

**Task 9**: Creating our Training Loop

**Task 10**: Loading and Evaluating our Model

## Task 1: Introduction

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)

<img src="Images/BERT_diagrams.pdf" width="1000">

## Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

import os
from google.colab import drive

In [2]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
downloaded = drive.CreateFile({'id':'1YzeLky763-yFYCuN_cWTVrkZxN1OmW-i'}) 
downloaded.GetContentFile('smile-annotations-final.csv')
df = pd.read_csv('smile-annotations-final.csv',
                 names = ['id','text','category'])
df.set_index('id',inplace=True)

In [4]:
df.text.iloc[0]

'happy'

In [5]:
df.category.value_counts()

nocode               1537
happy                1110
not-relevant          209
angry                  56
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [6]:
df = df.iloc[1:]
df = df[~df.category.str.contains('\|')]

In [7]:
df = df[df.category != 'nocode']

In [8]:
df.category.value_counts()

happy           1110
not-relevant     209
angry             56
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [9]:
possible_labels = df.category.unique()

In [10]:
label_dict = {}
for index,possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [11]:
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [12]:
df['label'] = df.category.replace(label_dict)
df.head(10)

,text,category,label
id,,,
614460759154798597,A week writing reports about reports suddenly ...,happy,0
611230763531522048,La fama viene y va... @arteblanco @NationalGal...,not-relevant,1
612252317241446400,Samba time! @britishmuseum @camdentalking #Dis...,happy,0
613057647990824960,@NationalGallery The 2nd GENOCIDE against #Bia...,not-relevant,1
613677567619563520,Lots of ancient artefacts to enjoy @britishmus...,happy,0
611844583224438784,Plus excellent prizes from the @britishmuseum ...,happy,0
615216447787270144,"Feliz cumpleaños,Rubens! Happy birthday,Rubens...",happy,0
610741907426267136,. @NationalGallery What message does oil spons...,angry,2
613634413730078720,MPs are to debate @NationalGallery privatisati...,not-relevant,1


## Task 3: Training/Validation Split

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train,x_val,y_train,y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    random_state=17,
    stratify=df.label.values
)

In [15]:
df['data_type'] = ['not_set']*df.shape[0]

In [16]:
df.head()

,text,category,label,data_type
id,,,,
614460759154798597,A week writing reports about reports suddenly ...,happy,0,not_set
611230763531522048,La fama viene y va... @arteblanco @NationalGal...,not-relevant,1,not_set
612252317241446400,Samba time! @britishmuseum @camdentalking #Dis...,happy,0,not_set
613057647990824960,@NationalGallery The 2nd GENOCIDE against #Bia...,not-relevant,1,not_set
613677567619563520,Lots of ancient artefacts to enjoy @britishmus...,happy,0,not_set


In [17]:
df.loc[x_train, 'data_type'] = 'train'
df.loc[x_val, 'data_type'] = 'val'

In [18]:
df.groupby(['category', 'label','data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           8
disgust      3     train         5
                   val           1
happy        0     train       943
                   val         167
not-relevant 1     train       177
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

## Task 4: Loading Tokenizer and Encoding our Data

In [19]:
pip install transformers

     |████████████████████████████████| 2.3MB 27.5MB/s 
     |████████████████████████████████| 3.3MB 43.6MB/s 
     |████████████████████████████████| 901kB 28.2MB/s 


In [20]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [21]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True)

In [22]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_specila_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256
    ,return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_specila_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256
    ,return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Keyword arguments {'add_specila_tokens': True} not recognized.
Keyword arguments {'add_specila_token

In [23]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val,
                        labels_val)

In [24]:
len(dataset_train)

1230

In [25]:
len(dataset_val)

218

## Task 5: Setting up BERT Pretrained Model

In [26]:
from transformers import BertForSequenceClassification

In [27]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                      num_labels=len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states=False
                                     )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Task 6: Creating Data Loaders

In [28]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [29]:
batch_size = 32

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

## Task 7: Setting Up Optimizer and Scheduler

In [30]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [31]:
optimizer = AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8
)

In [32]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs
    
)

## Task 8: Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [33]:
import numpy as np

In [34]:
from sklearn.metrics import f1_score

In [35]:
#pred = [0.9,0.05,0.05,0.0]

In [36]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis =1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [37]:
def accuracy_per_class(preds, labels):
    labels_dict_inverse = {v: k for k,v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis =1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'class : {labels_dict_inverse[label]}')
        print(f'accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')

## Task 9: Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [38]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [39]:
device = torch.device('cuda' if torch.cuda.is_available()  else 'cpu')
model.to(device)

print(device)

cuda


In [40]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [45]:
from google.colab import drive
drive.mount('/content/drive')

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    loss_train_total = 0
    
    progress_bar=tqdm(dataloader_train,
                      desc='Epoch {:1d}'.format(epoch),
                     leave=False,
                     disable=False)
    
    for batch in progress_bar:
        
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids'       : batch[0],
            'attention_mask' : batch[1],
            'labels'          : batch[2]
        }
        
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss' : '{:.3f}'.format(loss.item()/len(batch))})

    torch.save(model.state_dict(), f'/content/drive/My Drive/BERT Sentiment Analysis SMILE Twitter Data/Models/BERT_ft_epoch{epoch}.model')
               
    tqdm.write(f'\nEpoch : {epoch}')
               
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



 Epoch : 1
Training loss: 0.2466401186085294



Validation loss: 0.47558034530707766
F1 Score (weighted): 0.8670266785339358



 Epoch : 2
Training loss: 0.13449126285199958



Validation loss: 0.5007164584738868
F1 Score (weighted): 0.8898712651432106


...


Validation loss: 0.6255212287817683
F1 Score (weighted): 0.8937402544183769



 Epoch : 9
Training loss: 0.014974763700451291



Validation loss: 0.6156096890169594
F1 Score (weighted): 0.8937402544183769



 Epoch : 10
Training loss: 0.014166343185934238



Validation loss: 0.6361256444028446
F1 Score (weighted): 0.8937402544183769



## Task 10: Loading and Evaluating our Model

In [46]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [47]:
model.to(device)
pass

In [48]:
model.load_state_dict(
torch.load('/content/drive/My Drive/BERT Sentiment Analysis SMILE Twitter Data/Models/BERT_ft_epoch10.model',
          map_location=torch.device('cpu'))
)

<All keys matched successfully>

In [49]:
_, predictions, true_vals = evaluate(dataloader_val)

In [50]:
accuracy_per_class(predictions, true_vals)

class : happy
accuracy: 162/167
class : not-relevant
accuracy: 22/32
class : angry
accuracy: 5/8
class : disgust
accuracy: 0/1
class : sad
accuracy: 1/5
class : surprise
accuracy: 5/5


In [ ]:
# Google Colab -- GPU Instance (K80)
#batch_size = 32
# epoch = 10